In [16]:
import streamlit as st
import plotly.express as px
import pandas as pd
import plotly.io as pio
import glob

from mainnet_launch.pages.exit_liquidity.estimate_exit_liquidity_from_quotes import (
    fetch_and_render_exit_liquidity_from_quotes,
)
from mainnet_launch.constants import AUTO_ETH

pio.templates.default = None

quote_df, slippage_df2 = fetch_and_render_exit_liquidity_from_quotes(AUTO_ETH, None, None)
slippage_df2

fetch_raw_amounts_by_destination took 0.7620 seconds.


,symbol,sell_amount_norm,buy_amount_norm,token_price,reference_quantity,token_price_at_reference_quantity,highest_sold_amount,percent_sold,bps_loss_excess_vs_reference
0,ETHx,5.000000,5.320097,1.064019,5,NaN,985.512286,0.51,NaN
1,ETHx,98.551229,104.824564,1.063656,5,NaN,985.512286,10.00,NaN
2,ETHx,985.512286,1046.727660,1.062115,5,NaN,985.512286,100.00,NaN
3,WETH,5.000000,5.000000,1.000000,5,NaN,3672.450423,0.14,NaN
4,WETH,367.245042,367.245042,1.000000,5,NaN,3672.450423,10.00,NaN
5,WETH,3672.450423,3672.450423,1.000000,5,NaN,3672.450423,100.00,NaN
6,iETHv2,4.314280,5.052426,1.171094,5,NaN,43.142800,10.00,NaN
7,iETHv2,5.000000,5.855468,1.171094,5,NaN,43.142800,11.59,NaN
8,iETHv2,43.142800,50.524150,1.171091,5,NaN,43.142800,100.00,NaN
9,osETH,5.000000,5.252298,1.050460,5,NaN,2052.015828,0.24,NaN


### This is a method for estimating the excess slippage from moving at size to exit one of our assets. 

- Get a `reference price` by selling 10k (quantity) of a stable coin, or or 5 quantity of a LST / LRT for the base asset.

For example if we sell 

5 stETH and the best quote we get is for 4.9 ETH

then the reference price is 4.9 ETH / 5 stETH = 0.98 ETH/stETH

By checking the quotes at larger sizes we can estimate the excess slippage

For example

Selling 100 stETH -> 97.5 ETH then the reference price = 97.5

Therefore the excess slippage is

10000 * (0.98 - 0.975) / 0.98 = 51bps

This can help inform if we are too exposed for an asset. 

### Other notes
- This uses `buyAmount` instead of `minBuyAmount` to in all calculations.
- This looks at the quantity of assets we hold, then sells a percent of them. 
- For stable coins we also include a check for `[50k, 100k and 200k]` quantity of the stablecoin
- This is delibertly slow, because of rate limiting in the swapper API. It uses sevearl different dex aggregators that each have different rate limits. In the past when I didn't make is this slow, some tokens would look like they would lose 50 -90% of the value on the larger sales. This went away after slowing down the speed of requests for quotes. 
- To avoid outliers, this makes 3 different requests for the same, (with a 12, 24, 36) second delay between them then reports the median.
- This doesn't save data, everything is fetched live.

### Know issues
- This does not take into account that we might be trading against ourselves when we exit an asset. For example, if we are most of the pxETH:ETH liqudity, then by selling a large amount of pxETH, this method 
will give a quote that looks **better** that is actually would be because it is assuming that there is this large pool we could trade into. 

# Estimating Excess Slippage on Asset Exits
This method helps quantify how much extra slippage we incur when selling larger chunks of an asset.

1. Reference Price

- Execute a small “reference” sale

- For stablecoins: sell 10 000 units

- For LSTs/LRTs: sell 5 units

Compute the reference price
- Example: sell 5 stETH → receive 4.9 ETH
- Reference price = 4.9 ETH ÷ 5 stETH = 0.98 ETH/stETH

2. Measuring Excess Slippage

- Sell a larger quantity (e.g., 100 stETH → receive 97.5 ETH)
- New price = 97.5 ETH ÷ 100 stETH = 0.975 ETH/stETH

- Excess slippage in basis points (bps):

 `slippage_bps = 10 000 × (0.98 – 0.975) ÷ 0.98 ≈ 51 bps`

- This tells you how far the large-sale price has fallen relative to your reference.

3. Key Details

- The quote data source is our swapper API at https://swaps-pricing.tokemaklabs.com/swap-quote-v2.

- Use buyAmount (not minBuyAmount) in all calculations.

- Percent-based scaling: looks at the current balance across each autopool and sells a percentage of it.

- Additional stablecoin checks at quantities [50 000, 100 000, 200 000].

- Deliberately slow: Because of various DEX-aggregator rate limits we need to be slower to avoid spurious 50–90 % “losses” on large sales.

- Outlier mitigation: for each size, perform three quotes (with 12 s, 24 s, and 36 s delays) and report the median.

- No data is saved: all data is fetched live each run.

4. Known Issues
If we are a large share of the pool (e.g. most of pxETH:ETH liquidity), the large-sale quote can look artificially better because in the real world we would be effectively trading against ourselves.



,symbol,sell_amount_norm,buy_amount_norm,token_price,reference_quantity,token_price_at_reference_quantity,highest_sold_amount,percent_sold,bps_loss_excess_vs_reference
0,ETHx,5.000000,5.320097,1.064019,5,1.064019,985.512286,0.51,0.000000
1,ETHx,98.551229,104.824564,1.063656,5,1.064019,985.512286,10.00,3.419991
2,ETHx,985.512286,1046.727660,1.062115,5,1.064019,985.512286,100.00,17.927099
3,WETH,5.000000,5.000000,1.000000,5,1.000000,3672.450423,0.14,0.000000
4,WETH,367.245042,367.245042,1.000000,5,1.000000,3672.450423,10.00,0.000000
5,WETH,3672.450423,3672.450423,1.000000,5,1.000000,3672.450423,100.00,0.000000
6,iETHv2,4.314280,5.052426,1.171094,5,1.171094,43.142800,10.00,-0.000394
7,iETHv2,5.000000,5.855468,1.171094,5,1.171094,43.142800,11.59,0.000000
8,iETHv2,43.142800,50.524150,1.171091,5,1.171094,43.142800,100.00,0.021995
9,osETH,5.000000,5.252298,1.050460,5,1.050460,2052.015828,0.24,0.000000


In [ ]:
slippage_df = (
    quote_df.groupby(["symbol", "sell_amount_norm"])[
        ["buy_amount_norm", "token_price", "sellAmount", "reference_quantity"]
    ]
    .first()
    .reset_index()
)

# slippage_df[['sellAmount', 'reference_quantity']].astype(int)

In [19]:
# slippage_df[['sellAmount', 'reference_quantity']]
#
#  = slippage_df[['sellAmount', 'reference_quantity']].astype(str)
# slippage_df[slippage_df["sellAmount"].astype(str) == slippage_df["reference_quantity"].astype(str)]

# slippage_df
# price_at_reference_quantity = (
#     slippage_df[slippage_df["sellAmount"].astype(str) == slippage_df["reference_quantity"].astype(str)]
#     .set_index("symbol")[["token_price"]]
#     .to_dict()
# )
# price_at_reference_quantity
# # highest_sold_amount = slippage_df.groupby("symbol")["sell_amount_norm"].max().to_dict()

# slippage_df["token_price_at_reference_quantity"] = slippage_df["symbol"].map(price_at_reference_quantity)
# slippage_df["highest_sold_amount"] = slippage_df["symbol"].map(highest_sold_amount)

# slippage_df["percent_sold"] = slippage_df.apply(
#     lambda row: round(100 * row["sell_amount_norm"] / row["highest_sold_amount"], 2), axis=1
# )
# slippage_df["bps_loss_excess_vs_reference"] = slippage_df.apply(
#     lambda row: 10_000 * (row["token_price_at_reference_quantity"] - row["token_price"]) / row["token_price"],
#     axis=1,
# )

In [22]:
px.scatter(
    slippage_df,
    x="sell_amount_norm",
    y="ratio",
    color="symbol",
    title="Sell amount to ratio sell / buy amount ratio",
)
# selling 1 is not a fair comparison because we some t

ValueError: Value of 'y' is not the name of a column in 'data_frame'. Expected one of ['symbol', 'sell_amount_norm', 'buy_amount_norm', 'token_price', 'reference_quantity', 'token_price_at_reference_quantity', 'highest_sold_amount', 'percent_sold', 'bps_loss_excess_vs_reference'] but received: ratio

In [ ]:
px.scatter(
    slippage_df,
    x="sell_amount_norm",
    y="buy_amount_norm",
    color="symbol",
    title="Sell amount to ratio sell / buy amount ratio",
)
# selling 1 is not a fair comparison because we some t

In [ ]:
1800 / 900

2.0

In [ ]:
usdt_df = df[df["symbol"] == "USDT"].copy()
usdt_df

,aggregatorName,asyncSwapper,internal,buyAmount,minBuyAmount,expiration,tx,fullQuoteDetails,chainId,systemName,...,token_address,chain_id,symbol,name,decimals,buy_amount_norm,min_buy_amount_norm,sell_amount_norm,ratio,min_buy_amount_ratio
30,Odos,0x8ea340AA42cD1f6e2471a5C4574EA62C9416B859,False,194730680594,193757027192,1.000000e+10,{'from': '0x0000000000000000000000000000000000...,{'quote': {'traceId': 'e72892a0-3333-4cd6-9895...,1,gen3,...,0xdAC17F958D2ee523a2206206994597C13D831ec7,1,USDT,Tether USD,6,194730.680594,193757.027192,194655.643823,1.000385,0.995384
31,0xV2,0x0AB82316aD6e206Cab38EF9D36bD8cAEa86Bc18B,False,389460126846,387512819400,1.000000e+10,{'from': '0x0000000000000000000000000000000000...,"{'blockNumber': '22840713', 'buyAmount': '3894...",1,gen3,...,0xdAC17F958D2ee523a2206206994597C13D831ec7,1,USDT,Tether USD,6,389460.126846,387512.819400,389311.287647,1.000382,0.995380
32,0xV2,0x0AB82316aD6e206Cab38EF9D36bD8cAEa86Bc18B,False,584188338766,581267388350,1.000000e+10,{'from': '0x0000000000000000000000000000000000...,"{'blockNumber': '22840713', 'buyAmount': '5841...",1,gen3,...,0xdAC17F958D2ee523a2206206994597C13D831ec7,1,USDT,Tether USD,6,584188.338766,581267.388350,583966.931471,1.000379,0.995377
33,Odos,0x8ea340AA42cD1f6e2471a5C4574EA62C9416B859,False,778915316378,775020739797,1.000000e+10,{'from': '0x0000000000000000000000000000000000...,{'quote': {'traceId': 'ef5d4d58-e6a0-4159-af93...,1,gen3,...,0xdAC17F958D2ee523a2206206994597C13D831ec7,1,USDT,Tether USD,6,778915.316378,775020.739797,778622.575295,1.000376,0.995374
34,0xV2,0x0AB82316aD6e206Cab38EF9D36bD8cAEa86Bc18B,False,973641059658,968772844750,1.000000e+10,{'from': '0x0000000000000000000000000000000000...,"{'blockNumber': '22840713', 'buyAmount': '9736...",1,gen3,...,0xdAC17F958D2ee523a2206206994597C13D831ec7,1,USDT,Tether USD,6,973641.059658,968772.844750,973278.219119,1.000373,0.995371
35,Odos,0x8ea340AA42cD1f6e2471a5C4574EA62C9416B859,False,1020390,1015289,1.000000e+10,{'from': '0x0000000000000000000000000000000000...,{'quote': {'traceId': 'ef6eb20d-8cab-4929-bd2b...,1,gen3,...,0xdAC17F958D2ee523a2206206994597C13D831ec7,1,USDT,Tether USD,6,1.020390,1.015289,1.000000,1.020390,1.015289


In [ ]:
# usdt quotes are bad

In [ ]:
df.columns

Index(['aggregatorName', 'asyncSwapper', 'internal', 'buyAmount',
       'minBuyAmount', 'expiration', 'tx', 'fullQuoteDetails', 'chainId',
       'systemName', 'slippageBps', 'taker', 'sellToken', 'buyToken',
       'sellAmount', 'includeSources', 'excludeSources', 'sellAll',
       'timeoutMS', 'transferToCaller', 'same_token', 'token_address',
       'chain_id', 'symbol', 'name', 'decimals', 'buy_amount_norm',
       'min_buy_amount_norm', 'sell_amount_norm', 'ratio',
       'min_buy_amount_ratio'],
      dtype='object')

In [ ]:
sUSDe_df[["buy_amount_norm", "sell_amount_norm", "ratio", "symbol", "buyToken", "sellToken"]]

,buy_amount_norm,sell_amount_norm,ratio,symbol,buyToken,sellToken
18,29861.360610,25263.017910,1.182019,sUSDe,0x865377367054516e17014CcdED1e7d814EDC9ce4,0x9D39A5DE30e57443BfF2A8307A4256c8797A3497
19,59721.915051,50526.035820,1.182003,sUSDe,0x865377367054516e17014CcdED1e7d814EDC9ce4,0x9D39A5DE30e57443BfF2A8307A4256c8797A3497
20,89581.545760,75789.053729,1.181985,sUSDe,0x865377367054516e17014CcdED1e7d814EDC9ce4,0x9D39A5DE30e57443BfF2A8307A4256c8797A3497
21,119440.492728,101052.071639,1.181970,sUSDe,0x865377367054516e17014CcdED1e7d814EDC9ce4,0x9D39A5DE30e57443BfF2A8307A4256c8797A3497
22,149299.353377,126315.089549,1.181960,sUSDe,0x865377367054516e17014CcdED1e7d814EDC9ce4,0x9D39A5DE30e57443BfF2A8307A4256c8797A3497
23,1.232677,1.000000,1.232677,sUSDe,0x865377367054516e17014CcdED1e7d814EDC9ce4,0x9D39A5DE30e57443BfF2A8307A4256c8797A3497
